## A short introduction to Reinforcement Learning

### 1) Part 1: implement Q-learning from scratch


We have discussed Q-learning during the class. As you know, it is an off-policy algorithm that uses the Time Difference $\delta_t$, which is the difference between the estimated value of $s_t$ and the better estimate $r_{t+1} + \gamma V^\pi (s_{t+1})$

$$ \delta_t = r_{t+1} + \gamma V^\pi (s_{t+1}) - V^\pi (s_t) $$

The general definition of Q-learning update rule is:

$$ Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha[ r_{t+1} + \gamma \max_a Q(s_{t+1}, a) - Q(s_t,a_t) ] $$


In this part, we are going to implement Q-learning in the simple setting of a 1D grid world:

![1D grid world](06_lab_1dgrid.png)

Make sure you understand:
- the size of the grid world (= number of states)
- the size of the action space (= number of possible actions)
- the size of the Q-table
- the expected reward for reaching each state

The first step will be to initialize an empty Q-table, a table of rewards, a move cost, and alpha and gamma parameters.

In [73]:
import numpy as np

# we have 2 actions : move left and move right
nb_action = 2
nb_state = 6

# we create a matrix 6*2 to represent the value of each move at a given state
QTable = np.zeros((nb_state,nb_action))

# the tab with the representation of the 6 states (-1 for the bad end, 1 for the good end, and 0 for other states)
reward = [-1,0,0,0,0,1 ]

# cost of one move
cost = 0.01

# learning rate - should not be too high, e.g. between .5 and .9
alpha = 0.9

# discount factor that shows how much you care about future (remember 0 for myopic)
gamma = 0.5

In [74]:
QTable[1]

array([0., 0.])

Now comes the interesting part. You need to write the main Q-learning loop.

The first version will simply iterate:
- choose an action (by looking up in the Q-table! Choose the most interesting move)
- move
- update the Q-table

When you get this version, you can make it more complete to add the exploration/exploitation with the $\epsilon$-greedy version, by initializing an $\epsilon = 1$ that you decrease by e.g. 0.01 in each iteration. In your main loop, start by drawing a random number. If it is lower that $\epsilon$, then EXPLORE (= take a random move), otherwise EXPLOIT (= choose the best move)

In [75]:
LEFT = 0
RIGHT = 1

In [127]:
# your code here
current_state = 2
def isStuked(reward, pos): return reward[pos] != 0

def computeNextState(pos, move):
  return pos + (1 if move else -1)

def loop(nb_epochs, table, cost, alpha, gamma, reward):
  stop = -1
  for i in range(nb_epochs):
    current_state = 2
    epsilon = 1
    nb_step = 0
    while True:
      epsilon = 0.5 - i * 0.001

      if np.random.random() < epsilon:

        move = 0 if np.random.random() > 0.5 else 1
        next_pos = computeNextState(current_state, move)

      else:

        move = 0 if table[current_state, 0] > table[current_state, 1] else 1
        next_pos = computeNextState(current_state, move)
        table[current_state, move] += alpha * (
            (reward[next_pos]-cost) + gamma * max(
                table[next_pos, 0], table[next_pos, 1]) - 
                table[current_state, move])
        
      nb_step += 1
      if nb_step == stop or isStuked(reward, next_pos):
        break
      current_state = next_pos

QTable = np.zeros((nb_state,nb_action))
loop(10000, QTable, cost, alpha, gamma, reward)
QTable

array([[ 0.     ,  0.     ],
       [-0.909  ,  0.69461],
       [-0.0099 ,  0.7829 ],
       [-0.01629,  0.881  ],
       [ 0.     ,  0.99   ],
       [ 0.     ,  0.     ]])

### 1) Part 2 (optional): FrozenLake with Open AI Gym

In this part, we will use [Gym](https://www.gymlibrary.dev) framework to play with Q-learning and SARSA (without implementing them).

You can browse the many example provided by Gym. We will use the [Frozen Lake](https://www.gymlibrary.dev/environments/toy_text/frozen_lake/#frozen-lake) game.

Frozen lake involves crossing a frozen lake from Start(S) to Goal(G) without falling into any Holes(H) by walking over the Frozen(F) lake. The agent may not always move in the intended direction due to the slippery nature of the frozen lake. Check the page for more information!

![Frozen lake](https://www.gymlibrary.dev/_images/frozen_lake.gif)

First, you'll need to install the library -- if not already installed.

In [113]:
try:
    import gym
except:
    !pip install gym
    import gym

Using Gym makes it very easy to implement RL algorithms.

For Frozen Lake, you'll need to initialize the environement with:

`env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True)`

As stated in this [nice blog](https://blog.paperspace.com/getting-started-with-openai-gym/), the basic structure of the environment is described by the `observation_space` and the `action_space` attributes of the Gym `Env` class.

The `observation_space` defines the structure as well as the legitimate values for the observation of the state of the environment. The observation can be different things for different environments. The most common form is a screenshot of the game. There can be other forms of observations as well, such as certain characteristics of the environment described in vector form.

Similarly, the `Env` class also defines an attribute called the `action_space`, which describes the numerical structure of the legitimate actions that can be applied to the environment.

Now, for the Frozen Lake, we will start by defining the environment, and initializing all needed variables.

In [205]:
env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True)

epsilon = 1
total_episodes = 1000
max_steps = 1000

lr_rate = 0.1
gamma = 0.9

Q = np.zeros((env.observation_space.n, env.action_space.n))

/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


We can now define a function that will choose an action given a state.

You can directly implement the $\epsilon$-greedy version. Note that `env.action_space.sample()` will select a ramdom action.

In [206]:
print([env.action_space.sample() for i in range(9)])

[0, 0, 3, 1, 0, 1, 0, 0, 1]


In [219]:
def choose_action(state):
  if np.random.random() < epsilon:
    return env.action_space.sample()
  else:
    return Q[state].argmax()

Now comes the Q-learning core learning part.

The following function will update the Q-table, given the inital state, the target state after moving, the reward, and the selected action.

In [220]:
def learnWithQLearning(state, state2, reward, action):
    # your code here
    Q[state, action] = alpha * (
            (reward-cost) + gamma * max(
                Q[state2, action], Q[state2, action]) - 
                Q[current_state, action]) + Q[state, action]

Note that we also can easily define the SARSA algorithm, very similar to Q-learning, that needs to know one step further: the next action.

(You may come back to this point later, and first finish the main iteration loop).

In [221]:
def learnWithSARSA(state, state2, reward, action, action2):
    # your code here
  ...

    

The main iteration loop is given below.

In [222]:
import pickle
Q = np.zeros((env.observation_space.n, env.action_space.n))
# Start
for episode in range(total_episodes):
    state = env.reset()
    t = 0
    #print("Episode ",episode)  
    epsilon = 1 
 
    while t < max_steps:
      # env.render(mode = "human")
      action = choose_action(state)  
      state2, reward, done, info = env.step(action)
      epsilon = 0.5 - episode * 0.001

      # your code here to choose an action and to update the table (= learn)
      
      learnWithQLearning(state, state2, reward, action)
      state = state2

      t += 1
      if done:
          break
      
      # Uncomment for better visualization
      # time.sleep(0.1)
    

print(Q)

# with open("frozenLake_qTable.pkl", 'wb') as f:
#     pickle.dump(Q, f)



[[ 3.35535731e+047  5.05651297e+159 -4.63223834e+028 -1.96163034e+054]
 [ 3.55470458e+021  7.39442559e+158 -6.55656196e+000 -1.47964194e+022]
 [-4.21645318e+019 -3.09576734e+018 -3.70010831e+000 -4.17688378e+011]
 [ 1.48246867e+032  2.68983399e+018  7.19963890e-002  1.14596031e+015]
 [ 1.30873057e+026  6.48044430e+080 -4.76221091e+013 -1.73212039e+024]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  0.00000000e+000]
 [-4.59728296e+019  2.73098356e+020 -4.13838399e+000  5.80472573e+012]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  0.00000000e+000]
 [ 9.55305600e+021  2.17618573e+051 -1.37157189e+012 -4.50179686e+015]
 [-2.33900748e+017 -3.39012571e+017  3.05948602e+012 -1.99040289e+015]
 [-8.82271899e+017 -1.04492488e+018  3.17581487e+011 -7.93673870e+014]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  0.00000000e+000]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  0.00000000e+000]
 [-1.74539607e+014 -2.33462333e+018  1.06785196e+013 -5.11214644e+014]
 [-2.1